## 概述

![](images/deepl-api-python.gif)

## 1. 电子书格式转换路径

首先，需要将电子书从 Kindle 中导出来，并用 ePubor 进行 deDRM，而后将电子书转换成 epub 文件。

我都是在 Amazon 上直接买，而后在电脑上安装一个老版本的 Kindle App，用鼠标右键点击书名，下载，并不打开该电子书，而后退出 Kindle。

[ePubor Ultimate](https://www.epubor.com/) 也是个收费软件，能把旧版 Kindle 下载的电子书的 DRM 去掉；将 `awz` 文件转换成 `epub` 文件。（可参考这个[网页](https://www.epubor.com/how-to-downgrade-kindle-for-pcmac.html)）

经过一段时间的尝试，还是使用 pandoc 转换 epub 到 html，相对于 Calibre 更干净一些。

安装 pandoc：

```bash
brew install pandoc
```

转换 epub 到 html：

```bash
pandoc --read=epub -t html --extract-media=images --wrap=none -o <htmlfilename> <epubfilename>
```

也可以在 `.bashrc` 或者 `.zshrc` 文件里编写以下函数：

```bash
pandoceh (){
    pandoc --read=epub -t html --extract-media=images --wrap=none -o $2 $1
}
```

而后使用以命令：

```bash
pandoceh <epubfilename> <htmlfilename>
```

## 2. 选择 html 格式作为翻译格式的原因

1. 可以保留书中大量的脚注、尾注及其链接；
2. DeepL 有专门的 API 参数处理 xml tag，`tag_handling="xml"`；
3. 可以通过 css 文件随意设置显示样式，比较灵活；
4. 可以通过插入 javascript 函数指定某种特定语言的显示（比如，只显示中文）；
5. 可以用来作为源文件转换成任意格式的电子书……

在调用 `tag_handling="xml"` 之后，DeepL API 返回的译文非常规整，能够保留所有 html tag；并且，“返回字符串” 与 “原字符串” 相同，可以作为一个判断依据 —— 该行有没有被翻译，如果没有，在生成的译文 html 文件中，该行没必要重复出现……

## 3. 使用 DeepL API 逐行提交返回翻译结果

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

# 若干需要使用的函数

def translate(text):
    result = requests.get( 
       "https://api.deepl.com/v2/translate",
       params={ 
         "auth_key": auth_key,
         "target_lang": target_language,
         "text": text,
         "tag_handling": "xml", # 这个参数确保 DeepL 正确处理 html tags
       },
    ) 
    return result.json()["translations"][0]["text"]

def add_language_tag(html, tag, classname):
    soup = BeautifulSoup(html)
    for the_tag in soup.find_all(tag):
        the_tag['class'] = the_tag.get('class', []) + [classname]
    return str(soup)

def write_into_file(filename, text):
    with open(filename, 'a', encoding='utf-8') as f:
        f.write("\n"+text)    

def zh_format(html):
    
    # 直双引号转换成弯双引号
    pttn = r'\s*"(.*?)\s*"'
    rpl = r'“\1”'
    html = re.sub(pttn, rpl, html)
    
    # 直单引号转换成弯单引号
    pttn = r"\s*'(.*?)\s*'"
    rpl = r'‘\1’'
    html = re.sub(pttn, rpl, html)
    
    # html tag 中被误伤的双直引号
    pttn = r'=[“”"](.*?)[“”"]'
    rpl = r'="\1"'
    html = re.sub(pttn, rpl, html)   
    
    # html tag 中被误伤的单直引号
    pttn = r"=[‘’'](.*?)[‘’']"
    rpl = r"='\1'"
    html = re.sub(pttn, rpl, html)
    
    # 弯引号之前的空格
    pttn = r'([\u4e00-\u9fa5])([“‘])'
    rpl = r'\1 \2'
    html = re.sub(pttn, rpl, html)

    # 弯引号之后的空格 —— 标点符号不在 \u4e00-\u9fa5 范围内
    pttn = r'([’”])([\u4e00-\u9fa5])'
    rpl = r'\1 \2'
    html = re.sub(pttn, rpl, html)
           
    # html tag: <i>, <em> 转换成 <strong>
    pttn = r'(<i |<em )'
    rpl = r'<strong '
    html = re.sub(pttn, rpl, html)
    
    # html tag: <i>, <em> 转换成 <strong>
    pttn = r'(/i>|/em>)'
    rpl = r'/strong>'
    html = re.sub(pttn, rpl, html)
    
    # html tag: strong 内部的 “”、‘’、《》、（）
    pttn = r'<strong (.*?)>([《（“‘]+)'
    rpl = r'\2<strong \1>'
    html = re.sub(pttn, rpl, html)
    
    pttn = r'([》）”’。，]+)</strong>'
    rpl = r'</strong>\1'
    html = re.sub(pttn, rpl, html)
    
    # 省略号
    pttn = r'(\. )+\s*。*\s*|。\s*(\. )+'
    rpl = r'…… '
    html = re.sub(pttn, rpl, html)   

    # 破折号
    pttn = r'&mdash；|&mdash;|--'
    rpl = r' —— '
    html = re.sub(pttn, rpl, html)
    
    # 姓名之间的 ·（重复三次）
    pttn = r'([\u4e00-\u9fa5])-([\u4e00-\u9fa5])'
    rpl = r'\1·\2'
    html = re.sub(pttn, rpl, html)
    
    pttn = r'([\u4e00-\u9fa5])-([\u4e00-\u9fa5])'
    rpl = r'\1·\2'
    html = re.sub(pttn, rpl, html)
    
    pttn = r'([\u4e00-\u9fa5])-([\u4e00-\u9fa5])'
    rpl = r'\1·\2'
    html = re.sub(pttn, rpl, html)

    # 姓名之间的 ·（中间含有一个英文字母的）
    pttn = r'([\u4e00-\u9fa5])-(.?)-([\u4e00-\u9fa5])'
    rpl = r'\1·\2·\3'
    html = re.sub(pttn, rpl, html)

    # 全角百分号
    pttn = r'％'
    rpl = r'%'
    html = re.sub(pttn, rpl, html)
      
    # 数字前的空格
    pttn = r'([\u4e00-\u9fa5])(\d)'
    rpl = r'\1 \2'
    html = re.sub(pttn, rpl, html)
    
    # 数字后的空格，百分比 % 后的空格
    pttn = r'([\d%])([\u4e00-\u9fa5])'
    rpl = r'\1 \2'
    html = re.sub(pttn, rpl, html)
        
    # 英文字母前的空格
    pttn = r'([\u4e00-\u9fa5])([a-zA-Z])'
    rpl = r'\1 \2'
    html = re.sub(pttn, rpl, html)
        
    # 英文字母后的空格，百分比 % 后的空格
    pttn = r'([a-zA-Z])([\u4e00-\u9fa5])'
    rpl = r'\1 \2'
    html = re.sub(pttn, rpl, html)
        
    # 弯引号前的逗号
    pttn = r'，([”’])'
    rpl = r'\1，'
    html = re.sub(pttn, rpl, html)
        
    # 中文标点符号之前多余的空格
    pttn = r'([，。！？》〉】]) '
    rpl = r'\1'
    html = re.sub(pttn, rpl, html)
    
    # 英文句号 . 与汉字之间的空格
    pttn = r'\.([\u4e00-\u9fa5])'
    rpl = r'. \1'
    html = re.sub(pttn, rpl, html)
      
    # 左半角括号
    pttn = r'\s*\('
    rpl = r'（'
    html = re.sub(pttn, rpl, html)
    
    # 右半角括号
    pttn = r'\)\s*'
    rpl = r'）'
    html = re.sub(pttn, rpl, html)  

    # 多余的括号（DeepL 返回文本经常出现的情况）
    pttn = r'）。）'
    rpl = r'。）'
    html = re.sub(pttn, rpl, html)
    
    return html

# 指定一些变量

auth_key = "<your api auth_key>" # 注意，要订阅的是 DeepL API Pro
target_language = "ZH"  ## 当然，你可以将目标语言设置成任何 DeepL 支持的语言

# tags_to_be_translated = ['p', 'h1', 'h2', 'h3', 'h4']
tags_to_be_translated = ['p', 'h1', 'h2', 'h3', 'h4']
tags_tbt = '|'.join(tags_to_be_translated) # 为了以后在 regular expression 中使用

path  = "/Users/joker/Calibre Library/craft of research/" # 文件夹名称末尾得有 /
source_filename = "index.html"  # 用 pandoc 转换生成的文件，成为这一步的 “源文件”
                                # pandoc --read=epub -t html --extract-media=images --wrap=none -o epubfilename htmlfilename
                                # 在 .bashrc 或者 .zshrc 文件中可写函数：
                                # pandoceh (){
                                #    pandoc --read=epub -t html --extract-media=images --wrap=none -o $2 $1
                                # }
                                # 而后在 terminal 里使用 pandoceh epubfilename htmlfilename
                        
target_filename = "index-en-zhcn.html"

lines = open(path+source_filename, "r").readlines()

new_lines = []
line_count = 0
# 指定从哪一行开始翻译
startline = 88
# 指定到哪一行停止翻译
endline = 3683
# 是否是重新尝试
retry = 0

# 开始逐行处理

for line in lines:
    
    line_count += 1
    print(line_count)
    
    if (line_count < startline) or (line_count > endline):
        new_lines.append(line)
        print(line)
        if not retry:
            write_into_file(path+target_filename, line)
        continue  

    if line.strip() == '':
        new_lines.append(line)  
    
    if "<img" in line:
        write_into_file(path+target_filename, line)
        continue
    
    tags = [tag.name for tag in BeautifulSoup(line).find_all()]
    
    if len(tags) > 0 and line_count > startline and not retry:
        
        to_tranlate = False
        translating_tag = ""
        
        for tag in tags:
            if tag in tags_to_be_translated:
                to_tranlate = True
                translating_tag = tag
        
        if to_tranlate:
         
            succeeded = False
            while not succeeded:
                
                # 以下比较粗暴的 try... except，用来防止执行过程中出现 DeepL 连接错误而导致翻译任务中断……

                soupline = BeautifulSoup(line, 'html.parser')
                    
                line = str(soupline)
                
                try:
                    line_translated = translate(line)

                    # 以下一行确保将返回的字符串转换成一整行，而非含有 \n 的多行文本
                    line_translated = line_translated.replace("\n", "")
                    succeeded = True
                except:
                    succeeded = False
        
            line = add_language_tag(line, translating_tag, 'english')
            line_translated = add_language_tag(zh_format(line_translated), translating_tag, 'chinese')

            new_lines.append(line)
            print(line)
            write_into_file(path+target_filename, line)      

            new_lines.append(line_translated)
            print(line_translated)
            write_into_file(path+target_filename, line_translated + '\n')

        else:
            new_lines.append(line)
            print(line)
            write_into_file(path+target_filename, line)
            continue
      
            
print ('finished!')            
